#### SMA :
$SMA(n,t)= \frac{1}{n} \sum_{i=1}^{n-1} X_{t-i}$


In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime
import plotly.io as pio
pio.renderers.default = "browser"  # Ouvre le graphique dans le navigateur


In [58]:
df = pd.read_csv('C:\\Python\\Data_Viz Bitcoin in candlesticks\\Data\\Btc_Perp_USD.csv')

In [59]:
df

,timestamp,symbol,price,volume
0,2021-05-20T00:05:00.592Z,BTC-PERP,37049.0,0.0270
1,2021-05-19T00:05:00.615Z,BTC-PERP,42600.0,34.9140
2,2021-05-19T00:05:00.641Z,BTC-PERP,42600.0,35.2018
3,2021-05-19T00:05:00.692Z,BTC-PERP,42600.0,35.0053
4,2021-05-19T00:05:00.742Z,BTC-PERP,42600.0,35.0053
...,...,...,...,...
2132418,2021-05-20T00:05:00.474Z,BTC-PERP,37057.0,0.0230
2132419,2021-05-20T00:05:00.492Z,BTC-PERP,37057.0,0.0230
2132420,2021-05-20T00:05:00.517Z,BTC-PERP,37057.0,0.0230
2132421,2021-05-20T00:05:00.545Z,BTC-PERP,37045.0,0.0788


In [60]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

# Simulation des données du cours Bitcoin
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Initialiser l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Visualisation Cours Bitcoin"

# Layout de l'application
app.layout = dbc.Container([
    html.H1("Visualisation du cours Bitcoin", className="text-center my-4"),
    
    # Filtres
    dbc.Row([
        dbc.Col([
            html.Label("Intervalle temporel", style={"font-weight": "bold"}),
            dcc.Dropdown(
                id='time-interval',
                options=[
                    {"label": "1 Minute", "value": "1T"},
                    {"label": "30 Minutes", "value": "30T"},
                    {"label": "1 Heure", "value": "1H"},
                    {"label": "24 Heures", "value": "1D"}
                ],
                value="1H",
                style={"border-radius": "5px"}
            )
        ], width=3),
        dbc.Col([
            html.Label("Type de moyenne mobile", style={"font-weight": "bold"}),
            dcc.Dropdown(
                id='ma-type',
                options=[
                    {"label": "Moyenne Simple", "value": "SMA"},
                    {"label": "Moyenne Exponentielle", "value": "EMA"}
                ],
                value="SMA",
                style={"border-radius": "5px"}
            )
        ], width=3),
        dbc.Col([
            html.Label("Plage de dates", style={"font-weight": "bold"}),
            dcc.DatePickerRange(
                id='date-range',
                start_date=df.index.min().date(),
                end_date=df.index.max().date(),
                display_format="DD-MM-YYYY"
            )
        ], width=4),
        dbc.Col([
            dbc.Button("Mode Clair/Sombre", id="theme-toggle", color="primary", className="mt-4"),
        ], width=2)
    ], className="mb-3"),
    
    # Graphique
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='bitcoin-chart', config={"displayModeBar": True}),
        ], width=12)
    ])
], fluid=True)

# Callbacks

# Callback pour mettre à jour le graphique
@app.callback(
    Output('bitcoin-chart', 'figure'),
    [Input('time-interval', 'value'),
     Input('ma-type', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_chart(time_interval, ma_type, start_date, end_date):
    # Filtrer les données selon les dates
    filtered_df = df.loc[start_date:end_date]
    
    # Resampling des données
    resampled = filtered_df['price'].resample(time_interval).ohlc()
    
    # Calcul de la moyenne mobile
    if ma_type == "SMA":
        resampled['MA'] = resampled['close'].rolling(window=10).mean()
    elif ma_type == "EMA":
        resampled['MA'] = resampled['close'].ewm(span=10).mean()
    
    # Création du graphique
    fig = go.Figure(data=[
        go.Candlestick(
            x=resampled.index,
            open=resampled['open'],
            high=resampled['high'],
            low=resampled['low'],
            close=resampled['close'],
            name="Chandeliers"
        ),
        go.Scatter(
            x=resampled.index,
            y=resampled['MA'],
            mode='lines',
            name=f"{ma_type} (10 périodes)",
            line=dict(color="orange")
        )
    ])
    fig.update_layout(
        title="Graphique Bitcoin",
        xaxis_title="Temps",
        yaxis_title="Prix",
        template="plotly_dark" if theme == "dark" else "plotly",
    )
    return fig

# Callback pour gérer le mode sombre/clair
theme = "light"
@app.callback(
    Output('bitcoin-chart', 'config'),
    Input('theme-toggle', 'n_clicks')
)
def toggle_theme(n_clicks):
    global theme
    theme = "dark" if theme == "light" else "light"
    return {"template": "plotly_dark"} if theme == "dark" else {"template": "plotly"}

# Lancer le serveur
if __name__ == "__main__":
    app.run_server(debug=True)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[60], line 81, in update_chart(
    time_interval='1H',
    ma_type='SMA',
    start_date='2021-05-19',
    end_date='2021-05-20'
)
     72 @app.callback(
     73     Output('bitcoin-chart', 'figure'),
     74     [Input('time-interval', 'value'),
   (...)
     79 def update_chart(time_interval, ma_type, start_date, end_date):
     80     # Filtrer les données selon les dates
---> 81     filtered_df = df.loc[start_date:end_date]
        df =                                     symbol    price   volume
timestamp                                                   
2021-05-20 00:05:00.592000+00:00  BTC-PERP  37049.0   0.0270
2021-05-19 00:05:00.615000+00:00  BTC-PERP  42600.0  34.9140
2021-05-19 00:05:00.641000+00:00  BTC-PERP  42600.0  35.2018
2021-05-19 00:05:00.692000+00:00  BTC-PERP  42600.0  35.0053
2021-05-19 00:05:00.742000+00